**Import Libraries**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pylab as pylab
import cv2

import glob,os
from PIL import Image as im
import imageio
from tqdm import tqdm

import skimage
from skimage import io
from skimage import color
import skimage.filters
from skimage.filters import median
from skimage.morphology import disk
from skimage import exposure

from skimage.feature import canny
from skimage.filters import sobel

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Background and Artifact Removal**

In [ ]:
def read_img(filename):
    image = io.imread(filename)
    return image

In [ ]:
def arti_removal(filename):
    image = read_img(filename)
    gray_img = color.rgb2gray(image)
    hh,ww = gray_img.shape[:2]
    thresh = cv2.threshold(gray_img,0,255,cv2.THRESH_OTSU)[1] 
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    morph = cv2.morphologyEx(thresh,cv2.MORPH_CLOSE,kernel)

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    morph = cv2.morphologyEx(morph, cv2.MORPH_OPEN, kernel)

    contours = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0] if len(contours) == 2 else contours[1]
    big_contour = max(contours, key=cv2.contourArea)

    mask = np.zeros((hh,ww), dtype=np.uint8)
    cv2.drawContours(mask, [big_contour], 0, 255, cv2.FILLED)

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (55,55))
    mask = cv2.morphologyEx(mask, cv2.MORPH_DILATE, kernel)

    result = cv2.bitwise_and(img, img, mask=mask)
    
    return result

In [ ]:
def gray2bgr(image):
  bgr = color.gray2rgb(image)
  return bgr

In [ ]:
def resize(image):
  resized = cv2.resize(image, (256, 256))
  return resized

In [ ]:
def crop(image):  
      image_gray = color.rgb2gray(image)
      thresh = cv2.threshold(image_gray, 0 , 255, cv2.THRESH_BINARY)[1] 
      x, y, w, h = cv2.boundingRect(thresh) #find the bounding rectangle of nonzero points in the image
      crop_img = image[y:y+h, x:x+w,:] 
      bgr_crop = color.gray2rgb(image)
      return bgr_crop

In [ ]:
#psnr comparison between original and cropped image

**Pectoral Muscle Segmentation**

In [ ]:
def right_orient_mammogram(image):
    left_nonzero = cv2.countNonZero(image[:, 0:int(image.shape[1]/2)])
    right_nonzero = cv2.countNonZero(image[:, int(image.shape[1]/2):])
    
    if(left_nonzero < right_nonzero):
        image = cv2.flip(image, 1)

    return image

def read_image(image):
    #image = io.imread(filename)
    image = color.rgb2gray(image)
    image = right_orient_mammogram(image)
    image = exposure.equalize_hist(image)
    #image = median(image,disk(5))  #median isn't required for now
    #clahe = cv2.createCLAHE(clipLimit = 5)
    #image = clahe.apply(image) + 0
    return image

In [ ]:
def apply_canny(image):
    canny_img = canny(image, 3.05) #this value works with 90-95% of the images
    return sobel(canny_img)
    

In [ ]:
from skimage.transform import hough_line, hough_line_peaks

def get_hough_lines(canny_img):
    h, theta, d = hough_line(canny_img)
    lines = list()
    print('\nAll hough lines')
    for _, angle, dist in zip(*hough_line_peaks(h, theta, d)):
        print("Angle: {:.2f}, Dist: {:.2f}".format(np.degrees(angle), dist))
        x1 = 0
        y1 = (dist - x1 * np.cos(angle)) / np.sin(angle)
        x2 = canny_img.shape[1]
        y2 = (dist - x2 * np.cos(angle)) / np.sin(angle)
        lines.append({
            'dist': dist,
            'angle': np.degrees(angle),
            'point1': [x1, y1],
            'point2': [x2, y2]
        })
    
    return lines

In [ ]:
def shortlist_lines(lines):
    MIN_ANGLE = 10
    MAX_ANGLE = 70
    MIN_DIST  = 5
    MAX_DIST  = 256
    
    shortlisted_lines = [x for x in lines if 
                          (x['dist']>=MIN_DIST) &
                          (x['dist']<=MAX_DIST) &
                          (x['angle']>=MIN_ANGLE) &
                          (x['angle']<=MAX_ANGLE)
                        ]
    print('\nShorlisted lines')
    for i in shortlisted_lines:
        print("Angle: {:.2f}, Dist: {:.2f}".format(i['angle'], i['dist']))
        
    return shortlisted_lines

In [ ]:
from skimage.draw import polygon

def remove_pectoral(shortlisted_lines):
    shortlisted_lines.sort(key = lambda x: x['dist'])
    pectoral_line = shortlisted_lines[0]
    d = pectoral_line['dist']
    theta = np.radians(pectoral_line['angle'])
    
    x_intercept = d/np.cos(theta)
    y_intercept = d/np.sin(theta)
    
    return polygon([0, 0, y_intercept], [0, x_intercept, 0])

In [ ]:
#benign_pecless = []
#normal_pecless = []
malignant_pecless = []
def display_image(img):
    image = read_image(img)
    
    canny_image = apply_canny(image)
    lines = get_hough_lines(canny_image)
    shortlisted_lines = shortlist_lines(lines)

    fig, axes = plt.subplots(1, 4, figsize=(15,10))
    fig.tight_layout(pad=3.0)
    plt.xlim(0,image.shape[1])
    plt.ylim(image.shape[0])
    
    
    axes[0].set_title('Right-oriented mammogram')
    axes[0].imshow(image, cmap=pylab.cm.gray)
    axes[0].axis('on') 
    
    axes[1].set_title('Hough Lines on Canny Edge Image')
    axes[1].imshow(canny_image, cmap=pylab.cm.gray)
    axes[1].axis('on')
    axes[1].set_xlim(0,image.shape[1])
    axes[1].set_ylim(image.shape[0])
    for line in lines:
        axes[1].plot((line['point1'][0],line['point2'][0]), (line['point1'][1],line['point2'][1]), '-r')
        
    axes[2].set_title('Shortlisted Lines')
    axes[2].imshow(canny_image, cmap=pylab.cm.gray)
    axes[2].axis('on')
    axes[2].set_xlim(0,image.shape[1])
    axes[2].set_ylim(image.shape[0])
    for line in shortlisted_lines:
        axes[2].plot((line['point1'][0],line['point2'][0]), (line['point1'][1],line['point2'][1]), '-r')

        
    rr, cc = remove_pectoral(shortlisted_lines)
    try:
      image[rr, cc] = 0
    except Exception as e:
      return e
    #normal_pecless.append(image)
    malignant_pecless.append(image)
    #benign_pecless.append(image)
    plt.title('Pectoral muscle removed')
    plt.imshow(image,cmap='gray')
    
    axes[3].set_title('Pectoral muscle removed')
    axes[3].imshow(image, cmap=pylab.cm.gray)
    axes[3].axis('on') 
    
    plt.show() 

    return image

In [ ]:
def remove_graypx(image):
  binary_mask = image > 80
  restored = image.copy()
  restored[~binary_mask] = 0
  return restored

In [ ]:
padded_images = []
def padding(image):
  #image = io.imread(filename)
  image = cv2.cvtColor(image,cv2.COLOR_GRAY2RGB)
  #print(image.shape)
  old_image_h, old_image_w, channels = image.shape
  new_image_h = 256
  new_image_w = 256
  colour = (0,0,0)
  result_image = np.full((new_image_h,new_image_w,channels),colour,dtype=np.uint8)
  
  #center offset
  x_center = (new_image_w - old_image_w) // 2
  y_center = (new_image_h - old_image_h) // 2

  #copy image into center of resultant image
  result_image[y_center:y_center + old_image_h, x_center:x_center + old_image_w] = image  

  #resize image to 256x256
  resized = cv2.resize(result_image, (256, 256))


  padded_images.append(resized)

  return resized

In [ ]:
def preprocess(filename):
  img = read_img(filename)
  artiless_img = arti_removal(img)
  bgr_img = gray2bgr(artiless_img)
  resize_img = resize(bgr_img)
  crop_img = crop(resize_img)
  result_img = display_image(crop_img)
  nongray_img = remove_graypx(result_img)
  final_img = padding(nongray_img)
  return final_img






In [ ]:
images_directory = '/content/drive/MyDrive/ultrasound/images_u/malignant_cases'
pec_removed = '/content/drive/MyDrive/final_mini_mias/malignant'
for root, directory, file in os.walk(images_directory):
  for f in file:
    try:
      pec_removed_img = preprocess(root + '/' + f)
      preprocess(root + '/' + f)
      pec_removed_img_path = pec_removed + '/' + f
      #cv2.imwrite(pec_removed_img_path,255*pec_removed_img)
      imsave(pec_removed_img_path,pec_removed_img)
      print(pec_removed_img_path)
    except Exception as e:
      print('------------------------------')
      print('------------------------------')
      print('File that packed :',f)
      print('------------------------------')
      print('------------------------------')